Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [1]:
import string
import sys
import pandas as pd
from torch.utils.data import Dataset
import torch
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, Trainer, AutoModelForTokenClassification, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from character_tokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

data = pd.read_csv('all_accents.tsv', sep='\t', header=None, names=["word", "stressed_word"])
#model = AutoModelForTokenClassification.from_pretrained("IlyaGusev/ru-word-stress-transformer") 

#model_name = "DeepPavlov/rubert-base-cased"  
#model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2) 
#tokenizer = AutoTokenizer.from_pretrained(model_name)

#model_name = "microsoft/deberta-base"  # DeBERTa v2 базовая модель
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)

model=AutoModelForTokenClassification.from_pretrained('bert-base-uncased')
#tokenizer=AutoTokenizer.from_pretrained("KoichiYasuoka/bert-base-russian-upos")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

example = "Привет"
tokens = tokenizer(example)
print(tokens)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

C:\Users\baben_bakg1j1\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\baben_bakg1j1\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [2]:
print("Данные загружены:\n", data.head())

def get_stress_position(word, stressed_word):
    return stressed_word.index("^") - 1 if "^" in stressed_word else None

data['stress_position'] = data.apply(lambda row: get_stress_position(row['word'], row['stressed_word']), axis=1)
data = data.dropna(subset=['stress_position'])
data['stress_position'] = data['stress_position'].astype(int)

data = data.sample(n=50000, random_state=42)

train_words, test_words, train_labels, test_labels = train_test_split(
    data['word'].tolist(), data['stress_position'].tolist(), test_size=0.5, random_state=42
)

print("Пример из обучающей выборки:")
print("Слово:", train_words[2])
print("Позиция ударения:", train_labels[2])


Данные загружены:
       word stressed_word
0      -де          -д^е
1      -ка          -к^а
2    -либо        -л^ибо
3  -нибудь      -ниб^удь
4       -с            -с
Пример из обучающей выборки:
Слово: субсидирующие
Позиция ударения: 5


In [3]:
class StressDataset(Dataset):
    def __init__(self, words, labels, tokenizer, max_length=42):
        self.words = words
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.words)
    
    def __getitem__(self, idx):
        word = self.words[idx]
        label = self.labels[idx]
        
        tokens = self.tokenizer(word, padding="max_length", truncation=True, max_length=self.max_length)
        input_ids = tokens["input_ids"]
        
        label_ids = [-100] * len(input_ids)
        if 1 <= label < len(input_ids) - 1:
            label_ids[label + 1] = 1  
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(tokens["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(label_ids, dtype=torch.long),
        }

train_dataset = StressDataset(train_words, train_labels, tokenizer)
test_dataset = StressDataset(test_words, test_labels, tokenizer)

In [4]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return {
        'accuracy': accuracy_score(labels, preds)
    }

In [5]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    report_to=None 
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_accuracy']}")

C:\Users\baben_bakg1j1\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baben_bakg1j1\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Test Accuracy: 1.0


In [6]:
predictions = trainer.predict(test_dataset)

predicted_labels = predictions.predictions.argmax(-1)
true_labels = predictions.label_ids

random_indices = random.sample(range(len(test_dataset)), 10)

print("Слово - предсказание модели и реальная разметка:")
for idx in random_indices:
    word = test_words[idx]
    
    true_label = test_labels[idx]
    pred_label = None  

    for i, label in enumerate(predicted_labels[idx]):
        if label == 1:
            pred_label = i - 1  
            break

    print(f"Слово: {word}")
    print(f"  Предсказанная позиция ударения: {pred_label}")
    print(f"  Реальная позиция ударения: {true_label}")
    print()

Слово - предсказание модели и реальная разметка:
Слово: неотвязчивом
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 4

Слово: сбривался
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 4

Слово: истаивающих
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 2

Слово: протопившие
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 5

Слово: выбиваемая
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 4

Слово: пестрядинному
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 6

Слово: тезее
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 2

Слово: растаскивавший
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 3

Слово: заразившихся
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 4

Слово: тревожнейшие
  Предсказанная позиция ударения: -1
  Реальная позиция ударения: 3



In [7]:
def check_example(word, stress_position):
    tokens = tokenizer(word, padding="max_length", truncation=True, max_length=42)
    input_ids = tokens["input_ids"]
    
    label_ids = [-100] * len(input_ids)
    if 1 <= stress_position < len(input_ids) - 1:
        label_ids[stress_position + 1] = 1  
    
    decoded_word = tokenizer.decode(input_ids, skip_special_tokens=True)
    
    print(f"Слово: {word}")
    print(f"Токены: {tokens['input_ids']}")
    print(f"Метки: {label_ids}")
    print(f"Декодированное слово: {decoded_word}")
    print()

check_example("декламацию", 5)  
check_example("запутывалась", 2)  


Слово: декламацию
Токены: [0, 16, 18, 30, 32, 8, 34, 8, 54, 26, 70, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Метки: [-100, -100, -100, -100, -100, -100, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Декодированное слово: декламацию

Слово: запутывалась
Токены: [0, 24, 8, 40, 48, 46, 64, 12, 8, 32, 8, 44, 66, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Метки: [-100, -100, -100, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Декодированное слово: запутывалась

